In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, argparse, math, random
from datetime import datetime

testFiles = []
ka_test_file = "../dataset/prep/codebook/test_c0_"
kAA_test_file = "../dataset/prep/codebook/test_c1_"
kha_test_file = "../dataset/prep/codebook/test_c2_"

ka_train_file = "../dataset/prep/codebook/train_c0_"
kAA_train_file = "../dataset/prep/codebook/train_c1_"
kha_train_file = "../dataset/prep/codebook/train_c2_"

fileName = "../dataset/prep/codebook/train_c"

outputPath = "../output/abpi/"

# for every m and n combination (m = 2,4,8,16,32) and (n = 2,3,4,5)

N = [2,3,4,5]
M = [4,8,16,32]

# N = [2]
# M = [4]

wholeData = []

A = []
B = []
PI = []

In [2]:
def fileHandle(fileName):
    global wholeData
    wholeData = []
    file = open(fileName)
    for line in file:
        teLine = line.rstrip('\n ').split(' ')
        nLine = [int(i) for i in teLine]
        nLine = np.array(nLine)
#         print(nLine)
        wholeData.append(nLine)
    file.close()

def writeFile(classIndex, name, values, n, m, ext):
    targetPath = outputPath + name + "_c" + str(classIndex) + "_n" + str(n) + "_m" + str(m) + ext

    if not os.path.exists(os.path.dirname(targetPath)):
        try:
            os.makedirs(os.path.dirname(targetPath))
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    try:
        print("target File: ", targetPath)
        outfile = open(targetPath, "w")
    except IOError:
        print("File not created !!!!!!!!!!!!!!!!!!!!!!!!!")

    for value in values:
        for feature in value:
            outfile.write(str(feature)+" ")
        outfile.write("\n")
    outfile.close()
    print("Done !")

In [3]:
def EMAlgo(n, m):
    global A
    global B
    global PI
    global wholeData
    
    probI = 1e9
    probF = 0
    
    iterCount = 0
    while (abs(probI - probF) >= 1e-3) and (iterCount < 169):
        print("Iteration Number: ", iterCount, " ; probI: ",probI, " ; probF: ", probF, "xxxx: ",abs(probI - probF))
        iterCount += 1
        newA = np.zeros((n, n))
        newB = np.zeros((n, m))
        newPI = np.zeros((n))
        
#         print(B)
        
        # remind to send old - A, B in functions(forward, backward)
        # and calc of zeta, gamma
        # also remember to equate A = newA at the end of following for-loop!!
        
        probI = probF
        probF = 0
                
        for obs in wholeData:
            
            tempPLog = 0
            # From here, E-step start!!
            # See for alpha, beta to be set
            alpha = forwdProc(obs, n)
            beta = backwdProc(obs, n)
            
#             print(alpha)
            
            lObs = len(obs)
            
            for i in range(n):
                tempPLog += alpha[lObs-1,i]
            
            zeta = np.zeros((lObs-1, n, n))
                
            # pass the n as parameter in the func EMAlgo itself
            
            # common denominator
            deno = np.zeros((lObs-1))
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        deno[ti] += alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        zeta[ti,i,j] = alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        zeta[ti,i,j] /= deno[ti]
                        
            
            
            gamma = np.zeros((lObs, n))
            deno = np.zeros((lObs))
            
            for ti in range(lObs):
                for i in range(n):
                    deno[ti] += alpha[ti,i] * beta[ti,i]
                    
            
            for ti in range(lObs):
                for i in range(n):
                    gamma[ti,i] = alpha[ti,i] * beta[ti,i]
                    gamma[ti,i] /= deno[ti]
                    
            
            # E-step ends
            
            # M-step begins
            
            a = np.zeros((n, n))
            b = np.zeros((n, m))
            pi = np.zeros((n))
            
            for i in range(n):
                pi[i] = gamma[0,i]
                
            
            for i in range(n):
                denoGamma = 0
                nrtZeta = 0
                
                for ti in range(lObs-1):
                    denoGamma += gamma[ti,i]
                
                for j in range(n):
                    for ti in range(lObs-1):
                        nrtZeta += zeta[ti,i,j]
                    
                    a[i,j] = nrtZeta / denoGamma
                    
            
            # loop over the observation sequence!!

            for ti,syb in enumerate(obs):

                for j in range(n):

                    b[j,int(syb)] += gamma[ti,j]


            for j in range(n):
                tempVal = 0
                for ti in range(lObs):
                    tempVal += gamma[ti,j]

                for k in range(m):
                    b[j,k] /= tempVal


            newA += a
            newB += b
            newPI += pi
            

            probF += np.log(tempPLog)
            
        lData = len(wholeData)
        
#         print(lData)
        
        A = newA / lData
        B = newB / lData
        PI = newPI / lData
    

In [4]:
def initialize(n, m, classIndex):
    # call to EM from here!!
    # for each class loop
    # read the file from here itself for every class separately loop on the class itself 
    # outer loop for the class
    
    global A
    global B
    global PI
    global wholeData
    
    #treat them as global!!
    A = np.zeros((n,n))
    B = np.zeros((n,m))
    PI = np.zeros((n))
    
    for obs in wholeData:
        # obs is my numpy array
        
        a = np.zeros((n,n))
        b = np.zeros((n,m))
        
        lgt = len(obs)
        gone = int(lgt / n)
        toFirst = lgt - gone * n
        toFirst += gone
                
        for i in range(n-1):
            if i == 0:
                a[i,i] = (toFirst - 1) / toFirst
                a[i,i+1] = 1 / toFirst
            elif gone != 0:
                a[i,i] = (gone - 1) / gone
                a[i,i+1] = 1 / gone
        
        if gone != 0:
            a[n-1][n-1] = (gone - 1) / gone
            
        j = 0
        anth = 0
        for i,syb in enumerate(obs):
            if j == 0 and i < toFirst:
                b[0,int(syb)] += 1
                anth += 1
                continue
            elif j == 0:
                j = 1
                anth = 0
                
            if anth >= gone:
                j += 1
                anth = 0
            
            b[j,int(syb)] += 1
            anth += 1
            
        for i in range(n):
            if i == 0:
                b[i] /= toFirst
            elif gone != 0:
                b[i] /= gone
                
        A += a
        B += b
        
        
    lData = len(wholeData)
    A /= lData
    B /= lData
    PI[0] = 1
    
    #return from EMAlgo A, B, PI
    EMAlgo(n, m)    
    
    writeFile(classIndex, "em", A, n, m, ".a")
    writeFile(classIndex, "em", B, n, m, ".b")
    writeFile(classIndex, "em", [PI], n, m, ".pi")
    

In [5]:
def forwdProc(obs, n):
    
    global A
    global B
    global PI
    global wholeData
    
    # Assuming I know n
    lObs = len(obs)
    
    alpha = np.zeros((lObs, n))
    for i in range(n):
        alpha[0,i] = PI[i] * B[i,obs[0]]
        
    for ti in range(1,lObs):
        for j in range(n):
            storeVal = 0
            for i in range(n):
                storeVal += alpha[ti-1,i] * A[i,j]
            alpha[ti,j] = storeVal * B[j, obs[ti]]
    
    return alpha

In [6]:
def backwdProc(obs, n):
    # assuming I know n
    
    global A
    global B
    global PI
    global wholeData
    
    lObs = len(obs)
    beta = np.zeros((lObs, n))
    
    for i in range(n):
        beta[lObs-1,i] = 1
    
    for ti in range(lObs-2, -1, -1):
        for i in range(n):
            storeVal = 0
            for j in range(n):
                storeVal += A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
            beta[ti,i] = storeVal
    
    return beta

In [ ]:
for n in N:
    for m in M:
        # now read the files and store the data in wholeData
        # list of numpy arrays - wholeData[0] = one observation sequence, one numpy array
        print("M: ", m)
        for i in range(3):
            fileHandle(fileName+str(i)+"_"+ str(m) + ".cb")
            initialize(n,m,i);

M:  4
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -8756.578908607109 xxxx:  8756.578908607109
Iteration Number:  2  ; probI:  -8756.578908607109  ; probF:  -7161.801726314687 xxxx:  1594.7771822924224
Iteration Number:  3  ; probI:  -7161.801726314687  ; probF:  -7149.497750490181 xxxx:  12.30397582450587
Iteration Number:  4  ; probI:  -7149.497750490181  ; probF:  -7155.551433121385 xxxx:  6.053682631204538
Iteration Number:  5  ; probI:  -7155.551433121385  ; probF:  -7157.981950588854 xxxx:  2.430517467468235
Iteration Number:  6  ; probI:  -7157.981950588854  ; probF:  -7158.6414242080355 xxxx:  0.6594736191818811
Iteration Number:  7  ; probI:  -7158.6414242080355  ; probF:  -7158.8038675323905 xxxx:  0.16244332435508113
Iteration Number:  8  ; probI:  -7158.8038675323905  ; probF:  -7158.842268716721 xxxx:  0.03840118433072348
Iteration Number:  9  ; probI:  -7158.842268716721  ; probF:  -7158.8511506

Iteration Number:  1  ; probI:  0  ; probF:  -20377.871142501746 xxxx:  20377.871142501746
Iteration Number:  2  ; probI:  -20377.871142501746  ; probF:  -18805.794587499928 xxxx:  1572.0765550018186
Iteration Number:  3  ; probI:  -18805.794587499928  ; probF:  -18754.02043131325 xxxx:  51.77415618667874
Iteration Number:  4  ; probI:  -18754.02043131325  ; probF:  -18747.64856191098 xxxx:  6.371869402268203
Iteration Number:  5  ; probI:  -18747.64856191098  ; probF:  -18745.893094753832 xxxx:  1.7554671571488143
target File:  ../output/abpi/em_c0_n2_m16.a
Done !
target File:  ../output/abpi/em_c0_n2_m16.b
Done !
target File:  ../output/abpi/em_c0_n2_m16.pi
Done !
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -20377.871142501746 xxxx:  20377.871142501746
Iteration Number:  2  ; probI:  -20377.871142501746  ; probF:  -18805.794587499928 xxxx:  1572.0765550018186
Iteration Number:  3  ; probI:  -18805.79458749

Iteration Number:  39  ; probI:  -24729.76796047448  ; probF:  -24729.7665456683 xxxx:  0.0014148061818559654
Iteration Number:  40  ; probI:  -24729.7665456683  ; probF:  -24729.765502206305 xxxx:  0.0010434619944135193
target File:  ../output/abpi/em_c0_n2_m32.a
Done !
target File:  ../output/abpi/em_c0_n2_m32.b
Done !
target File:  ../output/abpi/em_c0_n2_m32.pi
Done !
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -26406.7146690914 xxxx:  26406.7146690914
Iteration Number:  2  ; probI:  -26406.7146690914  ; probF:  -24844.273332127123 xxxx:  1562.441336964279
Iteration Number:  3  ; probI:  -24844.273332127123  ; probF:  -24777.24587399985 xxxx:  67.02745812727153
Iteration Number:  4  ; probI:  -24777.24587399985  ; probF:  -24759.4320362423 xxxx:  17.813837757552392
Iteration Number:  5  ; probI:  -24759.4320362423  ; probF:  -24753.599359409738 xxxx:  5.832676832560537
Iteration Number:  6  ; probI:  -24

Iteration Number:  31  ; probI:  -43029.190149131035  ; probF:  -43029.806896788104 xxxx:  0.6167476570699364
Iteration Number:  32  ; probI:  -43029.806896788104  ; probF:  -43029.190603514886 xxxx:  0.6162932732186164
Iteration Number:  33  ; probI:  -43029.190603514886  ; probF:  -43029.80637437943 xxxx:  0.6157708645405364
Iteration Number:  34  ; probI:  -43029.80637437943  ; probF:  -43029.19105514688 xxxx:  0.6153192325436976
Iteration Number:  35  ; probI:  -43029.19105514688  ; probF:  -43029.805858086234 xxxx:  0.6148029393516481
Iteration Number:  36  ; probI:  -43029.805858086234  ; probF:  -43029.191503131486 xxxx:  0.6143549547487055
Iteration Number:  37  ; probI:  -43029.191503131486  ; probF:  -43029.805347223664 xxxx:  0.6138440921786241
Iteration Number:  38  ; probI:  -43029.805347223664  ; probF:  -43029.19194712969 xxxx:  0.6134000939709949
Iteration Number:  39  ; probI:  -43029.19194712969  ; probF:  -43029.804841471174 xxxx:  0.612894341480569
Iteration Number:

Iteration Number:  108  ; probI:  -43029.790173720656  ; probF:  -43029.20519316747 xxxx:  0.5849805531834136
Iteration Number:  109  ; probI:  -43029.20519316747  ; probF:  -43029.78980612874 xxxx:  0.5846129612691584
Iteration Number:  110  ; probI:  -43029.78980612874  ; probF:  -43029.20551518391 xxxx:  0.584290944832901
Iteration Number:  111  ; probI:  -43029.20551518391  ; probF:  -43029.78944167398 xxxx:  0.5839264900714625
Iteration Number:  112  ; probI:  -43029.78944167398  ; probF:  -43029.205834504195 xxxx:  0.5836071697849547
Iteration Number:  113  ; probI:  -43029.205834504195  ; probF:  -43029.78908032004 xxxx:  0.5832458158474765
Iteration Number:  114  ; probI:  -43029.78908032004  ; probF:  -43029.20615115881 xxxx:  0.582929161231732
Iteration Number:  115  ; probI:  -43029.20615115881  ; probF:  -43029.78872203123 xxxx:  0.5825708724223659
Iteration Number:  116  ; probI:  -43029.78872203123  ; probF:  -43029.20646517784 xxxx:  0.5822568533913
Iteration Number:  11

...

Iteration Number:  17  ; probI:  -28825.288614505178  ; probF:  -28833.361193663986 xxxx:  8.072579158808367
Iteration Number:  18  ; probI:  -28833.361193663986  ; probF:  -28842.070423632198 xxxx:  8.70922996821173
Iteration Number:  19  ; probI:  -28842.070423632198  ; probF:  -28851.310230415358 xxxx:  9.23980678316002
Iteration Number:  20  ; probI:  -28851.310230415358  ; probF:  -28860.986621696044 xxxx:  9.676391280685493
Iteration Number:  21  ; probI:  -28860.986621696044  ; probF:  -28870.95780702982 xxxx:  9.971185333775793
Iteration Number:  22  ; probI:  -28870.95780702982  ; probF:  -28880.973313002138 xxxx:  10.015505972318351
Iteration Number:  23  ; probI:  -28880.973313002138  ; probF:  -28890.600172429902 xxxx:  9.62685942776443
Iteration Number:  24  ; probI:  -28890.600172429902  ; probF:  -28899.146777694117 xxxx:  8.546605264215032
Iteration Number:  25  ; probI:  -28899.146777694117  ; probF:  -28905.709038070483 xxxx:  6.562260376365884
Iteration Number:  26  

Iteration Number:  93  ; probI:  -28981.62451147712  ; probF:  -28981.737281456113 xxxx:  0.11276997899403796
Iteration Number:  94  ; probI:  -28981.737281456113  ; probF:  -28981.84447664308 xxxx:  0.10719518696714658
Iteration Number:  95  ; probI:  -28981.84447664308  ; probF:  -28981.94638766103 xxxx:  0.10191101795135182
Iteration Number:  96  ; probI:  -28981.94638766103  ; probF:  -28982.04328750193 xxxx:  0.09689984089709469
Iteration Number:  97  ; probI:  -28982.04328750193  ; probF:  -28982.13543300471 xxxx:  0.09214550278193201
Iteration Number:  98  ; probI:  -28982.13543300471  ; probF:  -28982.223066149338 xxxx:  0.0876331446270342
Iteration Number:  99  ; probI:  -28982.223066149338  ; probF:  -28982.306415196294 xxxx:  0.08334904695584555
Iteration Number:  100  ; probI:  -28982.306415196294  ; probF:  -28982.38569569621 xxxx:  0.07928049991824082
Iteration Number:  101  ; probI:  -28982.38569569621  ; probF:  -28982.46111138881 xxxx:  0.07541569259774406
Iteration Nu

Iteration Number:  167  ; probI:  -28983.885633520495  ; probF:  -28983.888519606575 xxxx:  0.0028860860802524257
Iteration Number:  168  ; probI:  -28983.888519606575  ; probF:  -28983.89126678387 xxxx:  0.0027471772955323104
target File:  ../output/abpi/em_c2_n5_m16.a
Done !
target File:  ../output/abpi/em_c2_n5_m16.b
Done !
target File:  ../output/abpi/em_c2_n5_m16.pi
Done !
M:  32
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -25190.010304718286 xxxx:  25190.010304718286
Iteration Number:  2  ; probI:  -25190.010304718286  ; probF:  -21864.895235437823 xxxx:  3325.115069280462
Iteration Number:  3  ; probI:  -21864.895235437823  ; probF:  -21883.968099958325 xxxx:  19.072864520501753
Iteration Number:  4  ; probI:  -21883.968099958325  ; probF:  -21904.46135601724 xxxx:  20.493256058914994
Iteration Number:  5  ; probI:  -21904.46135601724  ; probF:  -21926.772777129223 xxxx:  22.31142111198278
Iteration N

Iteration Number:  72  ; probI:  -22088.574368068643  ; probF:  -22088.576062994074 xxxx:  0.0016949254313658457
Iteration Number:  73  ; probI:  -22088.576062994074  ; probF:  -22088.577523588978 xxxx:  0.0014605949036194943
Iteration Number:  74  ; probI:  -22088.577523588978  ; probF:  -22088.578785780683 xxxx:  0.0012621917048818432
Iteration Number:  75  ; probI:  -22088.578785780683  ; probF:  -22088.579879466433 xxxx:  0.001093685750674922
target File:  ../output/abpi/em_c0_n5_m32.a
Done !
target File:  ../output/abpi/em_c0_n5_m32.b
Done !
target File:  ../output/abpi/em_c0_n5_m32.pi
Done !
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -25190.010304718286 xxxx:  25190.010304718286
Iteration Number:  2  ; probI:  -25190.010304718286  ; probF:  -21864.895235437823 xxxx:  3325.115069280462
Iteration Number:  3  ; probI:  -21864.895235437823  ; probF:  -21883.968099958325 xxxx:  19.072864520501753
Iteration

Iteration Number:  71  ; probI:  -22088.572395520187  ; probF:  -22088.574368068643 xxxx:  0.0019725484562513884
Iteration Number:  72  ; probI:  -22088.574368068643  ; probF:  -22088.576062994074 xxxx:  0.0016949254313658457
Iteration Number:  73  ; probI:  -22088.576062994074  ; probF:  -22088.577523588978 xxxx:  0.0014605949036194943
Iteration Number:  74  ; probI:  -22088.577523588978  ; probF:  -22088.578785780683 xxxx:  0.0012621917048818432
Iteration Number:  75  ; probI:  -22088.578785780683  ; probF:  -22088.579879466433 xxxx:  0.001093685750674922
target File:  ../output/abpi/em_c1_n5_m32.a
Done !
target File:  ../output/abpi/em_c1_n5_m32.b
Done !
target File:  ../output/abpi/em_c1_n5_m32.pi
Done !
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -43411.62953600376 xxxx:  43411.62953600376
Iteration Number:  2  ; probI:  -43411.62953600376  ; probF:  -38411.41752023253 xxxx:  5000.212015771234
Iteration